In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("BigqueryExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Load data from BigQuery.
df_transfers = spark.read \
  .format("bigquery") \
  .load("de2024-436414.transfers_kaggle.transfers")    # project_id.datatset.tablename. Use your project id
df_transfers.printSchema()
df_transfers.show(4)

root
 |-- player_id: long (nullable = true)
 |-- transfer_date: date (nullable = true)
 |-- transfer_season: string (nullable = true)
 |-- from_club_id: long (nullable = true)
 |-- to_club_id: long (nullable = true)
 |-- from_club_name: string (nullable = true)
 |-- to_club_name: string (nullable = true)
 |-- transfer_fee: double (nullable = true)
 |-- market_value_in_eur: double (nullable = true)
 |-- player_name: string (nullable = true)

+---------+-------------+---------------+------------+----------+--------------+------------+------------+-------------------+-----------------+
|player_id|transfer_date|transfer_season|from_club_id|to_club_id|from_club_name|to_club_name|transfer_fee|market_value_in_eur|      player_name|
+---------+-------------+---------------+------------+----------+--------------+------------+------------+-------------------+-----------------+
|   671799|   2023-02-01|          22/23|        5411|       272|      FC Honka|         AIK|    300000.0|           500

In [ ]:
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Advanced_DF_EX1")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# load data
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path
gsc_file_path = 'gs://your_bucket/restaurants_geo.csv'  #  use your gcp bucket name. Also upload sales.csv first
# Create data frame
restaurants_geo_df = spark.read.format("csv").option("header", "true").option("delimiter", ";") \
       .load(gsc_file_path)
restaurants_geo_df.printSchema()
restaurants_geo_df.show(5)

restaurants_df = spark.read \
  .format("bigquery") \
  .load("your_project_id.labdataset.restaurants")    # project_id.datatset.tablename. Use your project id
restaurants_df.printSchema()
restaurants_df.show(4)

In [22]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Exercise_1_lab8_DE2")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Load data from BigQuery.

conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

gsc_file_path = 'gs://data_de2/transfers.csv'  #  use your gcp bucket name. Also upload sales.csv first

df_transfers = spark.read.format("csv").option("header", "true").option("delimiter", ",") \
       .load(gsc_file_path)
df_transfers.printSchema()
df_transfers.show(5)

df_players = spark.read \
  .format("bigquery") \
  .load("de2024-436414.transfers_kaggle.players")    # project_id.datatset.tablename. Use your project id
df_players.printSchema()
df_players.show(4)



root
 |-- player_id: string (nullable = true)
 |-- transfer_date: string (nullable = true)
 |-- transfer_season: string (nullable = true)
 |-- from_club_id: string (nullable = true)
 |-- to_club_id: string (nullable = true)
 |-- from_club_name: string (nullable = true)
 |-- to_club_name: string (nullable = true)
 |-- transfer_fee: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |-- player_name: string (nullable = true)

+---------+-------------+---------------+------------+----------+--------------+---------------+------------+-------------------+---------------+
|player_id|transfer_date|transfer_season|from_club_id|to_club_id|from_club_name|   to_club_name|transfer_fee|market_value_in_eur|    player_name|
+---------+-------------+---------------+------------+----------+--------------+---------------+------------+-------------------+---------------+
|   195778|   2026-06-30|          25/26|          79|        27| VfB Stuttgart|  Bayern Munich|       0.000|

In [12]:
df_players.columns

['player_id',
 'first_name',
 'last_name',
 'name',
 'last_season',
 'current_club_id',
 'player_code',
 'country_of_birth',
 'city_of_birth',
 'country_of_citizenship',
 'date_of_birth',
 'sub_position',
 'position',
 'foot',
 'height_in_cm',
 'contract_expiration_date',
 'agent_name',
 'image_url',
 'url',
 'current_club_domestic_competition_id',
 'current_club_name',
 'market_value_in_eur',
 'highest_market_value_in_eur']

**Which players hold the highest and second-highest transfer fees for each club, and what are their details?**

In [14]:
# first, only select the columns from df_players that are not in df_transfers and insightful
df_players_new = df_players.select("player_id", "player_code", "country_of_birth", "country_of_citizenship", 
                                   "date_of_birth", "sub_position", "position", "foot", "height_in_cm", 
                                   "agent_name", "image_url") 

df_players_new.show(15)



+---------+--------------------+----------------+----------------------+-------------------+------------+--------+-----+------------+----------------+--------------------+
|player_id|         player_code|country_of_birth|country_of_citizenship|      date_of_birth|sub_position|position| foot|height_in_cm|      agent_name|           image_url|
+---------+--------------------+----------------+----------------------+-------------------+------------+--------+-----+------------+----------------+--------------------+
|    56607|         anco-jansen|     Netherlands|           Netherlands|1989-03-09 00:00:00|        NULL| Missing|right|         186|            NULL|https://img.a.tra...|
|   231165|      fotis-doumanis|          Greece|                Greece|1993-10-28 00:00:00|        NULL| Missing|right|         173|            NULL|https://img.a.tra...|
|   727111|anastasios-papado...|            NULL|                Greece|2002-03-15 00:00:00|        NULL| Missing|right|         180|       

In [28]:
from pyspark.sql.functions import *
from pyspark.sql import Row, Window

joinExpression = df_players_new["player_id"] == df_transfers['player_id']
# default join is the inner join
players_merged = df_transfers.join(df_players_new, joinExpression,"left").drop("rid")
players_merged.show(20)

# Calculate the average fee per club
avg_fee_club = players_merged.groupBy("to_club_name").agg(avg("transfer_fee").alias("avg_fee_club"))

# Add the average fee to the players_merged DataFrame
players_with_avg_fee = players_merged.join(avg_fee_club, on="to_club_name", how="left")

# Rank the resturants in terms of the avg price, per each city. Then, select the best and worst product using ranks
highest_fee_club = Window.partitionBy(col("to_club_name")).orderBy(col("transfer_fee").desc())

players_merged_ranked = players_with_avg_fee.withColumn("rank_fee", dense_rank().over(highest_fee_club))

players_merged_ranked.show(20)
# Get the best and worst-performing restaurants
highest_players_table = players_merged_ranked.where((col("rank_fee") == 1).select("*"))
highest_players_table.show(20)

highest_players_table2 = highest_players_table.withColumn("transfer_fee", when(col("rank_desc") == 1, "Most Expensive")
                                                          .select("player_name", "to_club_name", "transfer_fee", "avg_price_club"))
cheap_expensive_city_table2.show(20)

+---------+-------------+---------------+------------+----------+---------------+---------------+------------+-------------------+-------------------+---------+-------------------+----------------+----------------------+-------------------+------------------+----------+-----+------------+---------------+--------------------+
|player_id|transfer_date|transfer_season|from_club_id|to_club_id| from_club_name|   to_club_name|transfer_fee|market_value_in_eur|        player_name|player_id|        player_code|country_of_birth|country_of_citizenship|      date_of_birth|      sub_position|  position| foot|height_in_cm|     agent_name|           image_url|
+---------+-------------+---------------+------------+----------+---------------+---------------+------------+-------------------+-------------------+---------+-------------------+----------------+----------------------+-------------------+------------------+----------+-----+------------+---------------+--------------------+
|   195778|   2026-

TypeError: 'Column' object is not callable

In [38]:
from pyspark.sql.functions import *
from pyspark.sql import Window

# Join the players and transfers datasets using a left join
join_expression = df_players_new["player_id"] == df_transfers["player_id"]
players_merged = df_transfers.join(df_players_new, join_expression, "left").drop("rid")

# We found some NAN values in the transfer fee section, we will remove these rows because they are not important
players_merged_clean = players_merged.filter(col("transfer_fee").isNotNull())

# Calculate the average fee per club for comparison
avg_fee_club = players_merged_clean.groupBy("to_club_name").agg(avg("transfer_fee").alias("avg_fee_club"))

# Add the average fee to the players_merged DataFrame
players_with_avg_fee = players_merged_clean.join(avg_fee_club, on="to_club_name", how="left")

# Define a window to rank players by transfer_fee within each club, ordering by transfer_fee in descending order
highest_fee_club = Window.partitionBy("to_club_name").orderBy(col("transfer_fee").desc(), col("player_name"))

# Add a rank column to determine the order of players by transfer_fee
players_merged_ranked = players_with_avg_fee.withColumn("rank_fee", row_number().over(highest_fee_club))


top_two_expenditures_table = players_merged_ranked.where((col("rank_fee") == 1) | (col("rank_fee") == 2) ).select("*")
top_two_expenditures_table.show(20)

top_two_expenditures_table2 = top_two_expenditures_table.withColumn("Record", when(col("rank_fee") == 1, "Most Expensive")
                                                          .otherwise("2nd Most expensive")).select("player_name", "to_club_name", "Record", "transfer_fee", "avg_fee_club")
top_two_expenditures_table2.show(20)


+---------------+---------+-------------+---------------+------------+----------+---------------+------------+-------------------+--------------------+---------+--------------------+--------------------+----------------------+-------------------+------------------+----------+-----+------------+------------------+--------------------+------------------+--------+
|   to_club_name|player_id|transfer_date|transfer_season|from_club_id|to_club_id| from_club_name|transfer_fee|market_value_in_eur|         player_name|player_id|         player_code|    country_of_birth|country_of_citizenship|      date_of_birth|      sub_position|  position| foot|height_in_cm|        agent_name|           image_url|      avg_fee_club|rank_fee|
+---------------+---------+-------------+---------------+------------+----------+---------------+------------+-------------------+--------------------+---------+--------------------+--------------------+----------------------+-------------------+------------------+-------